In [2]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import pandas as pd
import numpy as np
import os
import itertools
import random

url = 'https://raw.githubusercontent.com/BenBowring/Football/main/E0_21.csv'
data = pd.read_csv(url, index_col='Date')
data.index = pd.to_datetime(data.index)

home_labels = ['GF', 'GA', 'Opp']
home_cols = ['FTHG', 'FTAG', 'AwayTeam']

away_labels = ['GF', 'GA', 'Opp']
away_cols = ['FTAG', 'FTHG', 'HomeTeam']


matchups = [['Chelsea', 'West Brom'],
            ['Leicester', 'Man City'],
            ['Arsenal', 'Liverpool']]

# =============================================================================
# Intoduce weights for goals
# =============================================================================

# =============================================================================
# Make all below a class, then loop over each year of data to get stronger predictions
# =============================================================================

def get_poiss(data, matchups):
    
    teams = data['HomeTeam'].unique()
    
    dict_master = {}
    
    for team in teams:
    
        home_games = data[data['HomeTeam'] == team][home_cols]
        home_games.columns = home_labels
        home_games['H/A'] = 'H'
        
        away_games = data[data['AwayTeam'] == team][away_cols]
        away_games.columns = away_labels
        away_games['H/A'] = 'A'
    
        dict_master[team] = pd.concat([home_games, away_games], axis = 0).sort_index()
        
    
    df_total = pd.DataFrame({'Average Goals For': [dict_master[team]['GF'].mean() for team in teams],
                           'Average Goals Against': [dict_master[team]['GA'].mean() for team in teams],
                           'Predicted Goals For': [np.mean([dict_master[opp]['GA'].mean() for opp in dict_master[team]['Opp']]) for team in teams],
                           'Predicted Goals Against': [np.mean([dict_master[opp]['GF'].mean() for opp in dict_master[team]['Opp']]) for team in teams]}, 
                          index = teams)
    
    df_home = pd.DataFrame({'Average Goals For': [dict_master[team]['GF'][dict_master[team]['H/A'] == 'H'].mean() for team in teams],
                           'Average Goals Against': [dict_master[team]['GA'][dict_master[team]['H/A'] == 'H'].mean() for team in teams],
                           'Predicted Goals For': [np.mean([dict_master[opp]['GA'].mean() for opp in dict_master[team]['Opp'][dict_master[team]['H/A'] == 'H']]) for team in teams],
                           'Predicted Goals Against': [np.mean([dict_master[opp]['GF'].mean() for opp in dict_master[team]['Opp'][dict_master[team]['H/A'] == 'H']]) for team in teams]}, 
                          index = teams)
    
    df_away = pd.DataFrame({'Average Goals For': [dict_master[team]['GF'][dict_master[team]['H/A'] == 'A'].mean() for team in teams],
                           'Average Goals Against': [dict_master[team]['GA'][dict_master[team]['H/A'] == 'A'].mean() for team in teams],
                           'Predicted Goals For': [np.mean([dict_master[opp]['GA'].mean() for opp in dict_master[team]['Opp'][dict_master[team]['H/A'] == 'A']]) for team in teams],
                           'Predicted Goals Against': [np.mean([dict_master[opp]['GF'].mean() for opp in dict_master[team]['Opp'][dict_master[team]['H/A'] == 'A']]) for team in teams]}, 
                          index = teams)
    
    df_total['Adjusted Goals For'] = df_total.apply(lambda x: (x['Average Goals For']/x['Predicted Goals For']) * x['Average Goals For'], axis = 1)
    df_total['Adjusted Goals Against'] = df_total.apply(lambda x: (x['Average Goals Against']/x['Predicted Goals Against']) * x['Average Goals Against'], axis = 1) 
    
    df_home['Adjusted Goals For'] = df_home.apply(lambda x: (x['Average Goals For']/x['Predicted Goals For']) * x['Average Goals For'], axis = 1)
    df_home['Adjusted Goals Against'] = df_home.apply(lambda x: (x['Average Goals Against']/x['Predicted Goals Against']) * x['Average Goals Against'], axis = 1) 
    
    df_away['Adjusted Goals For'] = df_away.apply(lambda x: (x['Average Goals For']/x['Predicted Goals For']) * x['Average Goals For'], axis = 1)
    df_away['Adjusted Goals Against'] = df_away.apply(lambda x: (x['Average Goals Against']/x['Predicted Goals Against']) * x['Average Goals Against'], axis = 1) 
    
    
    # =============================================================================
    # Home vs Away Adjustment
    # =============================================================================
    
    df_adjust = pd.DataFrame({'Pred Home Goals For': [np.mean([df_home['Adjusted Goals For'].loc[x[0]], df_total['Adjusted Goals For'].loc[x[0]]]) for x in matchups],
                           'Pred Home Goals Against': [np.mean([df_home['Adjusted Goals Against'].loc[x[0]], df_total['Adjusted Goals Against'].loc[x[0]]]) for x in matchups],
                           'Pred Away Goals For': [np.mean([df_away['Adjusted Goals For'].loc[x[1]], df_total['Adjusted Goals For'].loc[x[1]]]) for x in matchups],
                           'Pred Away Goals Against': [np.mean([df_away['Adjusted Goals Against'].loc[x[1]], df_total['Adjusted Goals Against'].loc[x[1]]]) for x in matchups]})
    
    df_adjust.index = [x[0] + ' vs. ' + x[1] for x in matchups]
    
    # =============================================================================
    # Final df
    # =============================================================================
    
    df_out = pd.DataFrame({'Home Goals': df_adjust.apply(lambda x: np.mean([x['Pred Home Goals For'], x['Pred Away Goals Against']]), axis = 1),
                           'Away Goals': df_adjust.apply(lambda x: np.mean([x['Pred Away Goals For'], x['Pred Home Goals Against']]), axis = 1)})

    return(df_out)



In [3]:
get_poiss(data, matchups)

,Home Goals,Away Goals
Chelsea vs. West Brom,2.298230,0.703097
Leicester vs. Man City,1.433728,2.318099
Arsenal vs. Liverpool,1.147495,1.829595
